# Sales Forecasting With Machine Learning 

The purpose of this project is to demnostrate applied machine learning in the retail domain. 

Situation: Walmart launched an initative to crowd source machine learning models that could be used to forecast store sales. Walmart provided historical sales data, down to the department level, for 45 stores located in different regions. In addition, Walmart included promotional markdown event data. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. Walmart requested models prioritize forecasting accuracy during these markdown events despite comlete or ideal historical data.

Task: Predict the department-wide weekly sales for each store while prioritizing weeks with holiday markdown events.

Action: The following documnention reflects the process I followed to create a machine learning model that forecasts sales. 

## Libraries and Data Loading

First step was to import the necessary data and packages.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
from scipy.special import boxcox1p

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

features = pd.read_csv('/Users/aidan/Documents/Jupyter Storage/Walmart Forecasting/features.csv')
train = pd.read_csv('/Users/aidan/Documents/Jupyter Storage/Walmart Forecasting/train.csv')
stores = pd.read_csv('/Users/aidan/Documents/Jupyter Storage/Walmart Forecasting/stores.csv')
test = pd.read_csv('/Users/aidan/Documents/Jupyter Storage/Walmart Forecasting/test.csv')
sample_submission = pd.read_csv('/Users/aidan/Documents/Jupyter Storage/Walmart Forecasting/sampleSubmission.csv')


## Data Cleaning and Exploritory Analysis

In this section I explore the data sets provided, join pertinent information, and make relevant transformations. Also to establish context here is a description of each data set. 

####stores.csv
This file contains anonymized information about the 45 stores, indicating the type and size of store.

####train.csv
This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields

- Store - the store number
- Dept - the department number
- Date - the week
- Weekly_Sales -  sales for the given department in the given store
- IsHoliday - whether the week is a special holiday week

####test.csv
This file is identical to train.csv, except we have withheld the weekly sales. You must predict the sales for each triplet of store, department, and date in this file.

####features.csv
This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

- Store, the store number
- Date,  the week
- Temperature, average temperature in the region
- Fuel_Price, cost of fuel in the region
- MarkDown1-5, anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
- CPI, the consumer price index
- Unemployment, the unemployment rate
- IsHoliday, whether the week is a special holiday week

For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are identified in the data):
- Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
- Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
- Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
- Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13